In [1]:
from perekrestok_parser import PerekrestokScraper
from magnit_parser import MagnitScraper
import pandas as pd

import multiprocessing

In [2]:
def scrape_perekrestok():
    perekrestok_scraper = PerekrestokScraper()
    perekrestok_df = perekrestok_scraper.scrape_data()
    perekrestok_df.to_parquet('perekrestok.parquet')

def scrape_magnit():
    magnit_scraper = MagnitScraper()
    res_magnit = magnit_scraper.scrape_data()
    res_magnit.to_parquet('res_magnit.parquet')

if __name__ == '__main__':
    perekrestok_process = multiprocessing.Process(target=scrape_perekrestok)
    magnit_process = multiprocessing.Process(target=scrape_magnit)

    perekrestok_process.start()
    magnit_process.start()

    perekrestok_process.join()
    magnit_process.join()

[37277, 37651, 37653] was scrapped
[37655, 37657, 37659] was scrapped
{'cat_title': 'Ягоды', 'cat_url': 'https://www.perekrestok.ru/cat/c/154/agody'} was scraped
{'cat_title': 'Творожки', 'cat_url': 'https://www.perekrestok.ru/cat/c/657/tvorozki'} was scraped
[38699, 38705, 38707] was scrapped
{'cat_title': 'Bonte', 'cat_url': 'https://www.perekrestok.ru/cat/c/291/bonte'} was scraped
{'cat_title': 'Первые блюда', 'cat_url': 'https://www.perekrestok.ru/cat/c/31/pervye-bluda'} was scraped
{'cat_title': 'Начос', 'cat_url': 'https://www.perekrestok.ru/cat/c/774/nacos'} was scraped
[38715, 38711, 38721] was scrapped
{'cat_title': 'Меню на праздники', 'cat_url': 'https://www.perekrestok.ru/cat/c/1096/menu-na-prazdniki'} was scraped
[38391, 16869, 38555] was scrapped
[16849, 38393, 37465] was scrapped
[26995, 4854, 4840] was scrapped
{'cat_title': 'Kokoro', 'cat_url': 'https://www.perekrestok.ru/cat/c/292/kokoro'} was scraped
{'cat_title': 'Зелень и салаты', 'cat_url': 'https://www.perekresto

In [2]:
magnit_prod = pd.read_parquet('res_magnit.parquet')
magnit_cat = pd.read_parquet('magnit_categories.parquet')
perekrestok_prod = pd.read_parquet('perekrestok.parquet')

m1 = magnit_cat.iloc[:, :3].drop_duplicates()
m1.columns = ['cat_id', 'cat_name',  'cat_code']
m2 = magnit_cat.iloc[:, 3:6].drop_duplicates()
m2.columns = ['cat_id', 'cat_name',  'cat_code']
m3 = magnit_cat.iloc[:, 6:9].drop_duplicates()  
m3.columns = ['cat_id', 'cat_name',  'cat_code']
magnit_catgs = pd.concat([m1,m2,m3], ignore_index=True)

In [4]:
prod_magnit = pd.json_normalize(magnit_prod.explode('offers').to_dict(orient='records'))[['categories', 'code', 'grammar', 'id', 'isForAdults', 'name',
       'unitValue', 'offers.price',
       'offers.quantity']].explode('categories')
prod_magnit['offers.price'] = prod_magnit['offers.price'].str.replace(',','.').astype(float)
prod_cat_magnit = prod_magnit.merge(
    magnit_catgs[magnit_catgs.cat_name.notna()],
    how='left',
    left_on='categories',
    right_on='cat_id'
)

In [5]:
magnit_cat_avg = prod_cat_magnit.groupby('cat_name').agg({'offers.price':'mean'}).reset_index()
magnit_cat_avg.columns = ['category', 'avg_price']

In [8]:
perekrestok_prod['product_price'] = perekrestok_prod['product_price'].str.replace(',','.').str.split('₽').str[0].str.replace(' ','').astype(float)
perekrestok_cat_avg = perekrestok_prod.groupby('cat_title').agg({'product_price':'mean'}).reset_index()
perekrestok_cat_avg.columns = ['category', 'avg_price']

In [14]:
all_cats = perekrestok_cat_avg.merge(magnit_cat_avg, on='category', how='outer', suffixes=('_perekrestok', '_magnit'))
cats_mapped = all_cats[(all_cats.avg_price_perekrestok.isna()==False)&(all_cats.avg_price_magnit.isna()==False)]

In [37]:
perekrestok_unmapped = perekrestok_cat_avg[~perekrestok_cat_avg.category.isin(cats_mapped.category.values)]
magnit_unmapped = magnit_cat_avg[~magnit_cat_avg.category.isin(cats_mapped.category.values)]

In [103]:
from fuzzywuzzy import fuzz
#fuzz.partial_ratio('Автомасла', perekrestok_unmapped.category.values)

mapped_cats = []
for target_word in magnit_unmapped.category.values:
    target = 70

    best_match = None
    best_partial_ratio = 0
    best_matchs = {}
    for word in perekrestok_unmapped.category.values:
        #print(word)
        partial_ratio = fuzz.token_set_ratio(target_word, word)
        if partial_ratio > best_partial_ratio:
            #print(target_word, word,partial_ratio)
            best_partial_ratio = partial_ratio
            best_match = word
            #best_matchs[word] = best_partial_ratio

    if best_partial_ratio>=target:
        best_matchs['perekrestok'] = best_match
        best_matchs['magnit'] = target_word
    else:
        best_matchs['perekrestok'] = None
        best_matchs['magnit'] = target_word
    mapped_cats.append(best_matchs)


In [111]:
magnit_mapped = pd.DataFrame(mapped_cats).merge(magnit_unmapped, left_on='magnit', right_on='category', how='outer').rename({'avg_price':'avg_price_magnit'}, axis=1)
all_unmapped_mapped = magnit_mapped.merge(perekrestok_unmapped, left_on='perekrestok', right_on='category', how='outer').rename({'avg_price':'avg_price_perekrestok'}, axis=1)
#all_unmapped_mapped[(all_unmapped_mapped.avg_price_perekrestok.isna()==False)&(all_unmapped_mapped.avg_price_magnit.isna()==False)]

In [110]:

all_unmapped_mapped = all_unmapped_mapped[['magnit','avg_price_magnit','category_y','avg_price_perekrestok']].rename({'category_y':'perekrestok'}, axis=1)

cats_mapped['magnit'] = cats_mapped.category
cats_mapped['perekrestok'] = cats_mapped.category
cats_mapped = cats_mapped.drop('category', axis=1)

import numpy as np
all_mapped_categories = pd.concat([all_unmapped_mapped, cats_mapped])
all_mapped_categories['difference'] = np.where(all_mapped_categories['avg_price_magnit'].isna() | all_mapped_categories['avg_price_perekrestok'].isna(), 0, all_mapped_categories['avg_price_magnit'] - all_mapped_categories['avg_price_perekrestok'])
all_mapped_categories = all_mapped_categories.sort_values('difference', ascending=False).reset_index(drop=True)

/tmp/ipykernel_56460/2990299848.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_56460/2990299848.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,magnit,avg_price_magnit,perekrestok,avg_price_perekrestok,difference
0,Автомасла,1363.073333,NaN,NaN,0.000000
1,Автохимия,238.188430,NaN,NaN,0.000000
2,Аптечки дорожные,509.990000,NaN,NaN,0.000000
3,Бассейны,699.990000,NaN,NaN,0.000000
4,Безопасность,372.847143,NaN,NaN,0.000000
...,...,...,...,...,...
266,Фарш,161.823333,Фарш,234.241310,-72.417977
269,Фрукты,171.021053,Фрукты,288.945645,-117.924592
273,Хлебцы,92.444545,Хлебцы,99.534323,-7.089778
279,Чипсы,91.061429,Чипсы,140.026802,-48.965374


In [145]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd


df = all_mapped_categories

app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    dcc.Dropdown(
        id='magnit-category-dropdown',
        options=[{'label': category, 'value': category} for category in df['magnit'].dropna().unique()],
        multi=True,
        placeholder='Select magnit categories'
    ),
    html.Br(),
    dcc.Dropdown(
        id='perekrestok-category-dropdown',
        options=[{'label': category, 'value': category} for category in df['perekrestok'].dropna().unique()],
        multi=True,
        placeholder='Select perekrestok categories'
    ),
    html.Br(),
    html.Div([
        dcc.Input(
            id='avg-price-magnit-filter',
            type='number',
            placeholder='Filter by avg price (magnit)',
            debounce=True
        ),
        dcc.Dropdown(
            id='avg-price-magnit-operator',
            options=[
                {'label': '=', 'value': 'eq'},
                {'label': '>', 'value': 'gt'},
                {'label': '<', 'value': 'lt'},
            ],
            value='eq',
            clearable=False,
        ),
    ]),
    html.Br(),
    html.Div([
        dcc.Input(
            id='avg-price-perekrestok-filter',
            type='number',
            placeholder='Filter by avg price (perekrestok)',
            debounce=True
        ),
        dcc.Dropdown(
            id='avg-price-perekrestok-operator',
            options=[
                {'label': '=', 'value': 'eq'},
                {'label': '>', 'value': 'gt'},
                {'label': '<', 'value': 'lt'},
            ],
            value='eq',
            clearable=False,
        ),
    ]),
    html.Br(),
    html.Div([
        dcc.Input(
            id='difference-filter',
            type='number',
            placeholder='Filter by difference',
            debounce=True
        ),
        dcc.Dropdown(
            id='difference-operator',
            options=[
                {'label': '=', 'value': 'eq'},
                {'label': '>', 'value': 'gt'},
                {'label': '<', 'value': 'lt'},
            ],
            value='eq',
            clearable=False,
        ),
    ]),
    html.Br(),
    html.Table(id='filtered-table'),
])

@app.callback(
    Output('filtered-table', 'children'),
    Input('magnit-category-dropdown', 'value'),
    Input('perekrestok-category-dropdown', 'value'),
    Input('avg-price-magnit-filter', 'value'),
    Input('avg-price-magnit-operator', 'value'),
    Input('avg-price-perekrestok-filter', 'value'),
    Input('avg-price-perekrestok-operator', 'value'),
    Input('difference-filter', 'value'),
    Input('difference-operator', 'value')
)
def update_table(magnit_selected_categories, perekrestok_selected_categories, avg_price_magnit_filter,
                 avg_price_magnit_operator, avg_price_perekrestok_filter, avg_price_perekrestok_operator, difference_filter,
                 difference_operator):
    filtered_df = df.copy()

    if magnit_selected_categories:
        filtered_df = filtered_df[filtered_df['magnit'].isin(magnit_selected_categories)]
    if perekrestok_selected_categories:
        filtered_df = filtered_df[filtered_df['perekrestok'].isin(perekrestok_selected_categories)]
    if avg_price_magnit_filter is not None:
        if avg_price_magnit_operator == 'eq':
            filtered_df = filtered_df[filtered_df['avg_price_magnit'] == avg_price_magnit_filter]
        elif avg_price_magnit_operator == 'gt':
            filtered_df = filtered_df[filtered_df['avg_price_magnit'] > avg_price_magnit_filter]
        elif avg_price_magnit_operator == 'lt':
            filtered_df = filtered_df[filtered_df['avg_price_magnit'] < avg_price_magnit_filter]
    if avg_price_perekrestok_filter is not None:
        if avg_price_perekrestok_operator == 'eq':
            filtered_df = filtered_df[filtered_df['avg_price_perekrestok'] == avg_price_perekrestok_filter]
        elif avg_price_perekrestok_operator == 'gt':
            filtered_df = filtered_df[filtered_df['avg_price_perekrestok'] > avg_price_perekrestok_filter]
        elif avg_price_perekrestok_operator == 'lt':
            filtered_df = filtered_df[filtered_df['avg_price_perekrestok'] < avg_price_perekrestok_filter]
    if difference_filter is not None:
        if difference_operator == 'eq':
            filtered_df = filtered_df[filtered_df['difference'] == difference_filter]
        elif difference_operator == 'gt':
            filtered_df = filtered_df[filtered_df['difference'] > difference_filter]
        elif difference_operator == 'lt':
            filtered_df = filtered_df[filtered_df['difference'] < difference_filter]

    return [
        html.Tr([html.Th(col) for col in filtered_df.columns])] + [
        html.Tr([html.Td(filtered_df.iloc[i][col]) for col in filtered_df.columns]) for i in range(len(filtered_df))
    ]

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
#проблемы: не учитывается вес, размер, количество юнитов, нужно приводить к единому измерению (930мл превращать в 1000мл итд)
